# Beyond The Crosshair

In [7]:
import pandas as pd

## Datasets & Cleaning

### Counter-Strike

In [8]:
# Loading raw datasets
cs_data = pd.read_csv("../assets/raw/SteamDB Counter-Strike.csv")
css_data = pd.read_csv("../assets/raw/SteamDB Counter-Strike Source.csv")
cscz_data = pd.read_csv("../assets/raw/SteamDB Counter-Strike Condition Zero.csv")
cs2_data = pd.read_csv("../assets/raw/SteamDB Counter-Strike 2.csv")

In [9]:
fields_to_keep = ['month', 'peak', 'gain', '% gain'] # Fields we want to keep
list_of_cs_data = [cs_data, css_data, cscz_data, cs2_data] # Fields of datasets

# Cleaning the datasets in list_of_cs_data
for i in range(len(list_of_cs_data)):
    df = list_of_cs_data[i]
    df = df.rename(columns={"%gain": "% gain"})
    df = df.replace("-", "0")
    df[['peak', 'gain', '% gain']] = df[['peak', 'gain', '% gain']].replace({",": "", "%": ""}, regex=True).astype(float)
    df = df[fields_to_keep][1:] # Getting rid of first row of Last 30 days
    df["month"] = pd.to_datetime(df["month"], format="%b-%y")
    df = df.sort_values("month")
    list_of_cs_data[i] = df

# Splitting CS2 data into CS:GO and CS2
split_date = pd.Timestamp("2023-09-01")
csgo_data = list_of_cs_data[-1][list_of_cs_data[-1]["month"] < split_date]
list_of_cs_data[-1] = list_of_cs_data[-1][list_of_cs_data[-1]["month"] >= split_date]
list_of_cs_data.insert(-1, csgo_data)


In [10]:
# Merging the cleaned datasets
cs_merged = pd.concat(list_of_cs_data, ignore_index=True)
cs_merged = cs_merged[fields_to_keep]

In [11]:
# Grouping by month and summing the peak players
cs_merged_grouped = cs_merged.groupby(cs_merged.columns[0]).sum()

cs_merged_grouped = cs_merged_grouped.reset_index()
cs_merged_grouped = cs_merged_grouped.sort_values("month")

In [12]:
# Exporting clean datasets
list_of_cs_data[0].to_csv("../assets/clean/SteamDB_Counter-Strike_1.6_Clean.csv",encoding="utf-8", index=False, header=True)
list_of_cs_data[1].to_csv("../assets/clean/SteamDB_Counter-Strike_Source_Clean.csv",encoding="utf-8", index=False, header=True)
list_of_cs_data[2].to_csv("../assets/clean/SteamDB_Counter-Strike_Condition_Zero_Clean.csv",encoding="utf-8", index=False, header=True)
list_of_cs_data[3].to_csv("../assets/clean/SteamDB_Counter-Strike_Global_Offensive_Clean.csv",encoding="utf-8", index=False, header=True)
list_of_cs_data[4].to_csv("../assets/clean/SteamDB_Counter-Strike_2_Clean.csv",encoding="utf-8", index=False, header=True)
cs_merged_grouped.to_csv("../assets/clean/SteamDB_Counter-Strike_Clean.csv",encoding="utf-8", index=False, header=True)